# ERSAP-E2SAR localhost development

This notebook stands up a slice of 1 node and deploys E2SAR ERSAP segmentor/reassembly pipelines. This uses loopback to test the data exchange from segmentor and reassembler

## Preparation and overview

- Be sure to [generate a keypair for Jupyter Hub](GitHubSSH.ipynb) and register it with GitHub - the keys will be used to check out the code from private repositories, like [ERSAP-Java](https://github.com/JeffersonLab/ersap-java) and [E2SAR](https://github.com/JeffersonLab/E2SAR).
- Note that for E2SAR development and testing sender and receiver node compile/build environments will be setup via post-boot scripts ([sender](post-boot/sender.sh)) and grpc++ is installed as a tar.gz with static and dynamic libraries compiled for ubuntu22
- This does not setup the control plane node for anything, but testing a specific version - you can set which branch of UDPLBd to check out and a containerized version is built and stood up.

## Preamble

This cell must be executed whether you are creating a new slice or continuing work on the old one. If you are continuing work, you then skip the slice create section and proceed to wherever you left off.

In [12]:
#
# EDIT THIS
#
# if you want to force a site instead of using random
# Pick 'UCSD', 'SRI', 'FIU' or 'TOKY' - these sites have
# IPv4. Other sites use IPv6 management and have trouble
# retrieving git-lfs artifacts.
site_override = 'SRI'
#site_override = None

# GitHub SSH key file (private) registered using the GitHubSSH.ipynb notebook referenced above
github_key = '/home/fabric/work/fabric_config/github_ecdsa'

e2sar_branch = 'release' #there is no branch called release, this just keeps this notebooks slice separate.

#base distro type - either default or docker
distro_types = ['default','docker']
distro_type = distro_types[0]

# base distro 'ubuntu' or 'rocky'
distro_name = 'ubuntu'

#base distro version, currently only for ubuntu 20,22,24. E2SAR dependencies will be 
#downloaded for the appropriate versions.
distro_version = '22'

# note that the below is distribution specific ('ubuntu' for ubuntu and so on)
home_location = {
    'ubuntu': '/home/ubuntu',
    'rocky' : '/home/rocky'
}[distro_name]

vm_key_location = f'{home_location}/.ssh/github_ecdsa'

# which test suites in E2SAR to run (leave empty to run all)
# you can set 'unit' or 'live' to run unit or live tests only
e2sar_test_suite = ''

# name of the network connecting the nodes
net_name = 'site_bridge_net'

# url of e2sar deb. Find the appropriate version for the OS at https://github.com/JeffersonLab/E2SAR/releases
e2sar_branch = "main"
static_release_url = 'https://github.com/JeffersonLab/E2SAR/releases/download/E2SAR' # don't need to change this
e2sar_release_artifact = 'e2sar_0.1.3_amd64.deb'
e2sar_release_ver = '0.1.3'
e2sar_release_url = static_release_url + '-' + e2sar_branch + '-' + e2sar_release_ver + "-" + distro_name + "-" + distro_version + ".04/" + e2sar_release_artifact
print(e2sar_release_url)
#
# SHOULDN'T NEED TO EDIT BELOW
#
# Preamble
from datetime import datetime
from datetime import timezone
from datetime import timedelta

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

fablib = fablib_manager()             
fablib.show_config();

# Using docker image for cpnode by default
distro_image = distro_type + '_' + distro_name + '_' + distro_version
cp_distro_image = distro_types[1] + '_' + distro_name + '_' + distro_version

# variable settings
slice_name = f'ERSAP-E2SAR locahost test [{e2sar_branch}] on {distro_name}'

# for each node specify IP address (assuming /24), OS image
# note that most of the keys in these dictionaries map directly
# onto parameters to add_node()
node_config = {
    'sender': {
        'ip':'192.168.0.1', 
        'image': distro_image,
        'cores': 8,
        'ram': 24,
        'disk': 100 },
}
# skip these keys as they are not part of add_node params
skip_keys = ['ip']
# this is the NIC to use
nic_model = 'NIC_Basic'
# the subnet should match IPs
subnet = IPv4Network("192.168.1.0/24")

def execute_single_node(node, commands):
    for command in commands:
        print(f'\tExecuting "{command}" on node {node.get_name()}')
        #stdout, stderr = node.execute(command, quiet=True, output_file=node.get_name() + '_install.log')
        stdout, stderr = node.execute(command)
    if not stderr and len(stderr) > 0:
        print(f'Error encountered with "{command}": {stderr}')
        
def execute_commands(node, commands):
    if isinstance(node, list):
        for n in node:
            execute_single_node(n, commands)
    else:
        execute_single_node(node, commands)


https://github.com/JeffersonLab/E2SAR/releases/download/E2SAR-main-0.1.3-ubuntu-22.04/e2sar_0.1.3_amd64.deb


Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
Bastion Host,bastion.fabric-testbed.net
Bastion Username,srinivas_0000202712
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


## Create the slice

In [13]:
# list all slices I have running
output_dataframe = fablib.list_slices(output='pandas')
if output_dataframe:
    print(output_dataframe)
else:
    print('No active slices under this project')

ID,Name,Lease Expiration (UTC),Lease Start (UTC),Project ID,State
6dc09828-daba-472d-a6ed-87b0c62af3c1,3-node LB Tester Slice using ubuntu22,2024-10-22 14:15:35 +0000,2024-10-21 14:15:35 +0000,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca,StableOK


If your slice is already active you can skip to the 'Get Slice Details' section.

In [14]:
# List available images (this step is optional)
available_images = fablib.get_image_names()

print(f'Available images are: {available_images}')

Available images are: ['default_centos8_stream', 'default_centos9_stream', 'default_centos_7', 'default_centos_8', 'default_debian_10', 'default_debian_11', 'default_fedora_35', 'default_rocky_8', 'default_rocky_9', 'default_ubuntu_18', 'default_ubuntu_20', 'default_ubuntu_21', 'default_ubuntu_22', 'default_fedora_36', 'default_fedora_37', 'docker_rocky_8', 'docker_ubuntu_20', 'docker_ubuntu_22']


In [15]:
# find an available site in continental US
lon_west=-124.3993243
lon_east=-69.9721573

# getting a random site make take a bit of time
if not site_override:
    selected_site = fablib.get_random_site(filter_function=lambda x: x['location'][1] < lon_east
                                              and x['location'][1] > lon_west) 
else:
    selected_site = site_override

if selected_site:
    print(f'Selected site is {selected_site}')
else:
    print('Unable to find a site matching the requirements')

# write selected site into node attributes
for n in node_config:
    node_config[n]['site'] = selected_site
    

Selected site is SRI


In [17]:
# build a slice
slice = fablib.new_slice(name=slice_name)

# create a network
net1 = slice.add_l2network(name=net_name, subnet=subnet)

nodes = dict()
# create  nodes for sending and receiving with a selected network card
# use subnet address assignment
for node_name, node_attribs in node_config.items():
    print(f"{node_name=} {node_attribs['ip']}")
    nodes[node_name] = slice.add_node(name=node_name, **{x: node_attribs[x] for x in node_attribs if x not in skip_keys})
    nic_interface = nodes[node_name].add_component(model=nic_model, name='_'.join([node_name, nic_model, 'nic'])).get_interfaces()[0]
    net1.add_interface(nic_interface)
    nic_interface.set_mode('config')
    nic_interface.set_ip_addr(node_attribs['ip'])
    # postboot configuration is under 'post-boot' directory
    nodes[node_name].add_post_boot_upload_directory('post-boot','.')
    nodes[node_name].add_post_boot_execute(f'chmod +x post-boot/{node_name}.sh && ./post-boot/{node_name}.sh')

print(f'Creating a {distro_name} based slice named "{slice_name}" with nodes in {selected_site}')

# Submit the slice
slice.submit();


Retry: 8, Time: 309 sec


ID,902ac5eb-343d-4c73-9c23-a2ede1d23fe3
Name,ERSAP-E2SAR locahost test [main] on ubuntu
Lease Expiration (UTC),2024-10-22 16:17:36 +0000
Lease Start (UTC),2024-10-21 16:17:36 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
314b2b3a-532d-4f64-84f2-1bfad86f9172,sender,8,32,100,default_ubuntu_22,qcow2,sri-w3.fabric-testbed.net,SRI,ubuntu,192.5.67.87,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.87,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
9b27259f-288f-4152-a944-8fdc16144e46,site_bridge_net,L2,L2Bridge,SRI,192.168.1.0/24,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
sender-sender_NIC_Basic_nic-p1,p1,sender,site_bridge_net,100,config,,02:05:6F:4C:A6:FF,enp7s0,enp7s0,192.168.0.1,4,HundredGigE0/0/0/13



Time to print interfaces 311 seconds


## Get Slice Details

If not creating a new slice, and just continuing work on an existing one, execute this cell (in addition to the preamble) and then any of the cells below will work.

In [7]:
# get slice details (if not creating new)
slice = fablib.get_slice(name=slice_name)
a = slice.show()
nets = slice.list_networks()
nodes = slice.list_nodes()

sender = slice.get_node(name="sender")


# get node dataplane addresses
sender_addr = sender.get_interface(network_name=net_name).get_ip_addr()

sender_iface = sender.get_interface(network_name=net_name)

ID,4ff99358-2671-47f5-aba6-32c1568034ad
Name,ERSAP-E2SAR locahost test [main] on ubuntu
Lease Expiration (UTC),2024-10-22 15:36:00 +0000
Lease Start (UTC),2024-10-21 15:36:00 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
cc1a41d1-ff63-402f-bc6e-246f62e7a73e,site_bridge_net,L2,L2Bridge,SRI,192.168.1.0/24,None,Active,


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
9eb76414-e9cd-43d1-919c-6373c57e8a39,sender,8,32,100,default_ubuntu_22,qcow2,sri-w3.fabric-testbed.net,SRI,ubuntu,192.5.67.63,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.63,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


## Download and install E2SAR deb

In [8]:
# install github ssh key and set up build environment variables for interactive logins
commands = [
    f"chmod go-rwx {vm_key_location}",
    f"echo 'export LD_LIBRARY_PATH=/usr/local/lib' >> ~/.profile",
    f"echo 'export LD_LIBRARY_PATH=/usr/local/lib' >> ~/.bashrc",
]

for node in [sender]:    
    # upload the GitHub SSH key onto the VM
    result = node.upload_file(github_key, vm_key_location)
    execute_commands(node, commands)

	Executing "chmod go-rwx /home/ubuntu/.ssh/github_ecdsa" on node sender
	Executing "echo 'export LD_LIBRARY_PATH=/usr/local/lib' >> ~/.profile" on node sender
	Executing "echo 'export LD_LIBRARY_PATH=/usr/local/lib' >> ~/.bashrc" on node sender


In [9]:
#download boost and grpc dependencies from releases
commands = [
    f"wget -q -O e2sar-release.deb {e2sar_release_url}",
    f"sudo apt -yq install ./e2sar-release.deb",
]
 
execute_commands([sender], commands)

	Executing "wget -q -O e2sar-release.deb https://github.com/JeffersonLab/E2SAR/releases/download/E2SAR-main-0.1.3-ubuntu-22.04/e2sar_0.1.3_amd64.deb" on node sender
	Executing "sudo apt -yq install ./e2sar-release.deb" on node sender
Reading package lists... 

 
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  e2sar
0 upgraded, 1 newly installed, 0 to remove and 66 not upgraded.
Need to get 0 B/86.4 MB of archives.
After this operation, 464 MB of additional disk space will be used.
Get:1 /home/ubuntu/e2sar-release.deb e2sar amd64 0.1.3 [86.4 MB]
 debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
  debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-o

# Downloading ERSAP libraries

In [ ]:
commands = [
    f"echo 'export ERSAP_HOME=$HOME/ersap-install' >> ~/.profile",
    f"echo 'export ERSAP_HOME=$HOME/ersap-install' >> ~/.bashrc",
]
execute_commands([sender], commands)

In [2]:
commands = [
    f"mkdir ERSAP",
    f"cd ERSAP; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b upgradeGradle git@github.com:JeffersonLab/ersap-java.git",
    f"cd ERSAP; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b main git@github.com:JeffersonLab/ersap-cpp.git",
    f"cd ERSAP; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b main git@github.com:JeffersonLab/ersap-e2sar.git",
]
execute_commands([sender], commands)

NameError: name 'sender' is not defined

## Installing erap-java and ersap-cpp
ERSAP_HOME for this test is $HOME/ersap-install

In [ ]:
commands = [ 
    f"export ERSAP_HOME=$HOME/ersap-install; cd ERSAP/ersap-java; ./gradlew deploy",
    f"export ERSAP_HOME=$HOME/ersap-install; cd ERSAP/ersap-java; ./gradlew publishToMavenLocal",
]
execute_commands([sender], commands)

In [ ]:
commands = [ 
    f"export ERSAP_HOME=$HOME/ersap-install; cd ERSAP/ersap-cpp; ./configure --prefix='$ERSAP_HOME'",
    f"cd ERSAP/ersap-cpp; make install"
]
execute_commands([sender], commands)

## Installing ersap-e2sar Java and CPP actors

In [ ]:
commands = [ 
    f"export ERSAP_HOME=$HOME/ersap-install; cd ERSAP/ersap-e2sar/segmentor; ./gradlew install",
    f"export ERSAP_HOME=$HOME/ersap-install; cd ERSAP/ersap-e2sar/reassembler; ./gradlew install",
]
execute_commands([sender], commands)

In [ ]:
commands = [ 
    f"export ERSAP_HOME=$HOME/ersap-install; cd ERSAP/ersap-e2sar/segmentor; cmake -S . -B build",
    f"export ERSAP_HOME=$HOME/ersap-install; cd ERSAP/ersap-e2sar/reassembler; cmake -S . -B build",
]
execute_commands([sender], commands)

In [ ]:
commands = [ 
    f"export ERSAP_HOME=$HOME/ersap-install; cd ERSAP/ersap-e2sar/segmentor; cmake --build build --target install",
    f"export ERSAP_HOME=$HOME/ersap-install; cd ERSAP/ersap-e2sar/reassembler; cmake --build build --target install",
]
execute_commands([sender], commands)

## Running reassembler and segmentor localhost actors

In [ ]:
# deleting the output file in reassembler to make sure this works. Also printing out the input file

commands = [ 
    f"rm ERSAP/ersap-e2sar/reassembler/output/out_in.txt",
    f"echo 'input events:';cat ERSAP/ersap-e2sar/segmentor/input/in.txt",
]
execute_commands([sender], commands)

In [ ]:
import time
recv_command = f"export ERSAP_HOME=$HOME/ersap-install LD_LIBRARY_PATH=/usr/local/lib; cd ERSAP/ersap-e2sar/reassembler; $HOME/ersap-install/bin/ersap-shell config/localhost/reassembler_localhost.ersap"
send_command = f"export ERSAP_HOME=$HOME/ersap-install LD_LIBRARY_PATH=/usr/local/lib; cd ERSAP/ersap-e2sar/segmentor; $HOME/ersap-install/bin/ersap-shell config/localhost/segmentor_localhost.ersap"

# start the receiver for 10 seconds and log its output
print(f'Executing command {recv_command} on sender-reassembler')
sender.execute_thread(recv_command, output_file=f"reassembler-localhost.ersap-e2sar.log")

time.sleep(1)

# start the sender in the foreground
print(f'Executing command {send_command} on sender-segmentor')
stdout_send, stderr_send = sender.execute(send_command, output_file=f"{sender.get_name()}.perf.log")

print(f"Inspect reassembler-localhost.ersap-e2sar.log file in your Jupyter container to see the results")

In [ ]:
print("Output file contents")
# deleting the output file in reassembler to make sure this works. Also printing out the input file

commands = [ 
    f"cat ERSAP/ersap-e2sar/reassembler/output/out_in.txt",
]
execute_commands([sender], commands)

## Manage the slice

### Extend

In [ ]:
# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

### Delete

In [10]:
slice = fablib.get_slice(slice_name)
slice.delete()